In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

from src.data.features import split_train_test, split_features_target, make_document_vectors, make_day_feature
from src.data.features import sentiment_subjectivity
from src.data.features import generate_ents

from src.utils import load_dataset, home

raw = home / 'data' / 'interim'
raw = load_dataset(raw)
comb = raw['combined']
comb = comb.iloc[:, :]

comb.head(2)

In [ ]:
tr, te = split_train_test(comb)

In [ ]:
load = False
x_tr, y_tr = split_features_target(tr, 'train', load=load)
x_te, y_te = split_features_target(te, 'test', load=load)

## tidf

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from src.models.train import fit, rf_params, lr_params
from sklearn.feature_extraction.text import TfidfVectorizer

rf = fit(
    RandomForestClassifier(**rf_params),
    x_tr, y_tr, x_te, y_te, vec=TfidfVectorizer()
)

In [ ]:
rf = fit(
    LogisticRegression(**lr_params),
    x_tr, y_tr, x_te, y_te, vec=TfidfVectorizer()
)

## doc2vec

In [ ]:
tr_vecs, te_vecs = make_document_vectors(x_tr, x_te)

In [ ]:
rf = fit(
    RandomForestClassifier(**rf_params),
    tr_vecs, y_tr, te_vecs, y_te
)

## time series - day of week

In [ ]:
tr_dates, te_dates = make_day_feature(x_tr, x_te)

rf = fit(
    RandomForestClassifier(**rf_params),
    tr_dates, y_tr, te_dates, y_te
)

## Senitment & subjectivity

In [ ]:
x_s_tr = sentiment_subjectivity(x_tr)
x_s_te = sentiment_subjectivity(x_te)

rf = fit(
    RandomForestClassifier(**rf_params),
    x_s_tr.values, y_tr, x_s_te.values, y_te
)

## entity extraction 

In [ ]:
tr_ents = generate_ents(x_tr)
te_ents = generate_ents(x_te)

enc = CountVectorizer()
tr_ents = enc.fit_transform(tr_ents)
te_ents = enc.transform(te_ents)

rf = fit(
    RandomForestClassifier(**rf_params),
    tr_ents, y_tr, te_ents, y_te
)

In [ ]:
x_tr.head()

In [ ]:
#  tdif
vec = TfidfVectorizer()

tr_f = [
    vec.fit_transform(x_tr), 
]
te_f = [
    vec.transform(x_te), ]